In [1]:
### 接着处理数据集  生成自然语言描述的embedding
node_label_names = [ "account","company","person","medium"]

label_where_key = {
    "account":"nickname",
    "company":"companyName",
    "person":"personName",
    "medium":"mediumType",
}

import psycopg
import json
from pgvector.psycopg import register_vector
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import DirectoryLoader
import numpy as np

model_name = "/home/work/liuytest/demo/maidalun1020/bce-embedding-base_v1"
model_kwargs = {'device': 'npu:0'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

def find_real_prompt_from_pg_vector(user_query):
    pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
     
    pg_con.autocommit=True       
    cur = pg_con.cursor()
    register_vector(pg_con)

    embeddings = model.embed_documents(
        [
            user_query
        ]
    )
    embedding = embeddings[0]
    pg_con.execute('SET graph_path  = finbench;' )
    pg_con.execute('SET hnsw.ef_search = 100;' )
    embedding = np.asarray(embedding)
    datas = []

    prompt = "已知数据库中存在以下数据信息:\n"
    found = False
    for node in node_label_names:
        # pg_con.execute(f'drop  TABLE IF  EXISTS  zyykg_data_{node} cascade') match (n:person) return  n.feature_emd as emb  order by   (emb <#> %s)* -1 desc  LIMIT 1
        data = pg_con.execute(f'match (n:{node}) where (n.feature_emd::vector <#> %s) < -0.5  return  n ,(n.feature_emd::vector <#> %s) as score order by  score asc  LIMIT 5', (embedding,embedding,)).fetchall()
        if len(data)>0:
            found = True
            for data_inrow in data:
                prompt= prompt+f"得分为{data_inrow[1]},label为{data_inrow[0]},的节点\n"  
    if found:
        return prompt 
    else:
        return "" 



key_values = None
with open("tongyi_query_pair_query.json", 'r',encoding="utf-8") as json_file:
    key_values = json.load(json_file) 

new_query_pair_train = []
for row in key_values:
    new_row ={}
    for key,value in row.items():
        user_query = key
        data_prompt = find_real_prompt_from_pg_vector(user_query)
        print(user_query)
        print(data_prompt)
        new_row["instruction"] = data_prompt+"\n" + "请将以下自然语言翻译为对该数据库的Cypher查询:\n"+user_query
        new_row["output"]=value
        new_query_pair_train.append(new_row)
    if len(data_prompt) > 0:
        print("----------------------------------------------------------------------------------------------------")
        print(user_query)
        print(data_prompt)
        print("----------------------------------------------------------------------------------------------------")

with open('new_query_pair_train_tongyi.json', 'w',encoding="utf-8") as json_file:
    json.dump(new_query_pair_train, json_file ,ensure_ascii=False,indent=4)   

/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/latest owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")
/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packages/torch_npu/utils/path_manager.py:82: UserWarning: Warning: The /usr/local/Ascend/ascend-toolkit/8.0.RC1.alpha001/aarch64-linux/ascend_toolkit_install.info owner does not match the current user.
  warnings.warn(f"Warning: The {path} owner does not match the current user.")
/home/liuyang/miniconda3/envs/pytorch21/lib/python3.9/site-packag

查找昵称为Arlena Penha的账户的最后登录时间、是否被封禁和创建时间，最多返回20条结果
已知数据库中存在以下数据信息:
得分为-0.5038005113601685,label为account[3.1618]{"email": "hotmail.com", "nickname": "Arlena Penha", "phonenum": "487-8303", "accountId": "4614219293217785425", "isBlocked": "false", "accoutType": "brokerage account", "createTime": "2020-01-10 06:22:20.222", "accountLevel": "Basic level", "freqLoginType": "QRCode", "lastLoginTime": "1624940728902"},的节点

----------------------------------------------------------------------------------------------------
查找昵称为Arlena Penha的账户的最后登录时间、是否被封禁和创建时间，最多返回20条结果
已知数据库中存在以下数据信息:
得分为-0.5038005113601685,label为account[3.1618]{"email": "hotmail.com", "nickname": "Arlena Penha", "phonenum": "487-8303", "accountId": "4614219293217785425", "isBlocked": "false", "accoutType": "brokerage account", "createTime": "2020-01-10 06:22:20.222", "accountLevel": "Basic level", "freqLoginType": "QRCode", "lastLoginTime": "1624940728902"},的节点

-----------------------------------------------------------------